In [ ]:
from pytorch_pretrained_bert import BertTokenizer
from nltk.tokenize import sent_tokenize
from os import listdir
wiki_data_path = "../../../Data/Fever/wiki-pages/"
training_data = "../../../Data/Fever/train.jsonl"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import json
data = []
with open(training_data, "r") as f:
    lines = f.read().split("\n")
    for line in lines:
        data.append(json.loads(line))

In [ ]:
base_data = {}
ignored_lines = []
base_files = listdir(wiki_data_path)
for file in base_files:
    with open(wiki_data_path + file, "r") as f:
        lines = f.read().split("\n")
        for line in lines:
            try:
                _js = json.loads(line)
                base_data[_js["id"]] = _js
            except:
                ignored_lines.append(line)

In [ ]:
import re
from nltk.tokenize import sent_tokenize

def fix_text(text):
    text = text.replace(" -LRB- ", " (")
    text = text.replace(" -RRB- ", ") ")
    text = text.replace(" , ", ", ")
    text = text.replace(" . ", ". ")
    return text

processed_data = []

for line in data:
    try:
        claim = line["claim"]
        if line["verifiable"] == "VERIFIABLE":
            _all_evidence = {}
            for evidence in line["evidence"]:
                for evid in evidence:
                    title = evid[2]
                    if (title is not None):
                        if (title not in _all_evidence):
                            _all_evidence[title] = []
                        if (evid[3] not in _all_evidence[title]):
                            _all_evidence[title].append(evid[3])
            #print(_all_evidence)
            line["evidences"] = _all_evidence
            claim = tokenizer.tokenize(claim)
            evidentiary = []
            non_evidentiary = []
            for source in _all_evidence:
                parts = fix_text(base_data[source]["lines"]).split("\n")
                for i in range(len(parts)):
                    part = parts[i]
                    passage_parts = part.split("\t")
                    if (len(passage_parts[1]) > 0):
                        if (i in _all_evidence[source]):
                            evidentiary.append(tokenizer.tokenize(passage_parts[1]))
                        else:
                            non_evidentiary.append(tokenizer.tokenize(passage_parts[1]))
            line["processed"] = {
                "claim":claim, 
                "evidentiary":evidentiary,
                "non_evidentiary":non_evidentiary
            }
            processed_data.append(line)
    except:
        print("Ignoring line:", line)
        

In [ ]:
print(len(processed_data))

In [ ]:
import pickle
pickle.dump(processed_data, open("../fever_processed.pickle", "wb"))